# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [26]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product
import datetime

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament
from genetic_numpy import GA_discrete_actions, evaluate_fitness

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## Reading the Price data

In [27]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [29]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## MILP

In [30]:
milp_solver = MILP(plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"])

In [31]:
milp_model, milp_status, milp_profile = milp_solver.solve()

In [32]:
milp_status

'Optimal'

In [33]:
milp_model.objective.value()

350291.0

In [34]:
milp_profile.head()

,water_level,action,colour_id,utc_time,spot
0,0.0,0,nothing,2022-01-01 00:00:00+00:00,101.54
1,375.0,-1,pump,2022-01-01 01:00:00+00:00,52.13
2,750.0,-1,pump,2022-01-01 02:00:00+00:00,20.78
3,1125.0,-1,pump,2022-01-01 03:00:00+00:00,15.66
4,1500.0,-1,pump,2022-01-01 04:00:00+00:00,21.47


## Training

### Numpy Implementation

In [35]:
best_config = {
    "POP_SIZE": 500,
    "INITIAL_MUTATION_RATE": 0.7,
    "FINAL_MUTATION_RATE": 0.15,
    "INITIAL_EXPLORATION": 0.85,
    "ELITISM": 0.9,
    "SURVIVAL_RATE": 0.4,
}

ga_solver = GA_discrete_actions(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"], actions_space= [-1, 0, 1]
)

population, fitnesses, history = ga_solver.train(
    config=best_config,
    total_generations=500,
    tune_mode=False,
)

  0%|          | 0/500 [00:00<?, ?it/s]

In [36]:
history

,generation,best,average,worst
0,0,-345811.0,-2016579.946,-4664286.0
1,1,0.0,-1484281.164,-3422686.0
2,2,0.0,-1182136.752,-2956508.0
3,3,-13776.0,-1010483.566,-2997929.0
4,4,0.0,-901023.528,-2329237.0
...,...,...,...,...
495,495,309622.0,296256.406,-141501.0
496,496,309622.0,294194.916,-141731.0
497,497,309622.0,299202.396,-142275.0
498,498,309622.0,297419.910,-142275.0


In [37]:
history.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_training_history_elite_numpy.csv", index=False)

Best individual:

In [38]:
best_individual_df = df.assign(action = population[fitnesses.argmax()])
best_individual_df.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_best_individual_elite_numpy.csv", index=False)

In [39]:
best_individual_df

,spot,utc_time,action
0,101.54,2022-01-01 00:00:00+00:00,0.0
1,52.13,2022-01-01 01:00:00+00:00,1.0
2,20.78,2022-01-01 02:00:00+00:00,1.0
3,15.66,2022-01-01 03:00:00+00:00,1.0
4,21.47,2022-01-01 04:00:00+00:00,1.0
...,...,...,...
163,239.02,2022-01-07 19:00:00+00:00,0.0
164,223.79,2022-01-07 20:00:00+00:00,0.0
165,213.85,2022-01-07 21:00:00+00:00,0.0
166,211.64,2022-01-07 22:00:00+00:00,0.0


### Elite Selection

In [40]:
best_config = {
    "MUTPB": 1,
    "POP_SIZE": 200,
    "INITIAL_MUTATION_RATE": 0.05,
    "FINAL_MUTATION_RATE": 0.01,
    "INITIAL_EXPLORATION": 0.66,
    "ELITISM": 0.1,
}

In [41]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config=best_config,
    total_generations=500,
    tune_mode=False,
)

c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.


  0%|          | 0/500 [00:00<?, ?it/s]

In [42]:
history

,generation,best,average,worst
0,0,-187626.0,-1619343.915,-4437270.0
1,1,-100165.0,-876590.090,-2725694.0
2,2,-111323.0,-704760.850,-2044265.0
3,3,-119431.0,-606759.765,-1900917.0
4,4,-122540.0,-592313.495,-1757064.0
...,...,...,...,...
495,495,312026.0,156041.680,-332074.0
496,496,312026.0,173208.775,-247729.0
497,497,312026.0,173580.140,-328107.0
498,498,312026.0,164646.280,-428034.0


In [43]:
history.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_training_history_elite.csv", index=False)

Best individual:

In [44]:
best_individual_df = df.assign(action = population[np.array(fitnesses).argmax()])
best_individual_df.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_best_individual_elite.csv", index=False)

In [45]:
best_individual_df

,spot,utc_time,action
0,101.54,2022-01-01 00:00:00+00:00,0
1,52.13,2022-01-01 01:00:00+00:00,-1
2,20.78,2022-01-01 02:00:00+00:00,-1
3,15.66,2022-01-01 03:00:00+00:00,-1
4,21.47,2022-01-01 04:00:00+00:00,-1
...,...,...,...
163,239.02,2022-01-07 19:00:00+00:00,0
164,223.79,2022-01-07 20:00:00+00:00,0
165,213.85,2022-01-07 21:00:00+00:00,0
166,211.64,2022-01-07 22:00:00+00:00,0


### Tournament Selection

Take the results from tuning

In [46]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config={
        "MUTPB": 1.0,
        "POP_SIZE": 200,
        "CXPB": 0.65,
        "INITIAL_MUTATION_RATE": 0.05,
        "FINAL_MUTATION_RATE": 0.01,
        "INITIAL_EXPLORATION": 0.66,
        "TOURNAMENT_SIZE": 23,
    },
    total_generations=500,
    tune_mode=False,
)

c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.


  0%|          | 0/500 [00:00<?, ?it/s]

In [47]:
history

,generation,best,average,worst
0,0,-398543.0,-2074562.130,-4282401.0
1,1,-184117.0,-876430.295,-2016067.0
2,2,-165311.0,-544859.505,-1309331.0
3,3,-109810.0,-402093.465,-1346677.0
4,4,-80847.0,-371081.315,-970575.0
...,...,...,...,...
495,495,319679.0,214725.400,-234963.0
496,496,319679.0,200512.130,-428733.0
497,497,319679.0,196947.540,-442634.0
498,498,319679.0,189284.515,-438425.0


In [48]:
history.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_training_history_tournament.csv", index=False)

Best individual:

In [49]:
best_individual_df = df.assign(action = population[np.array(fitnesses).argmax()])
best_individual_df.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}_best_individual_tournament.csv", index=False)

In [50]:
best_individual_df

,spot,utc_time,action
0,101.54,2022-01-01 00:00:00+00:00,0
1,52.13,2022-01-01 01:00:00+00:00,-1
2,20.78,2022-01-01 02:00:00+00:00,-1
3,15.66,2022-01-01 03:00:00+00:00,-1
4,21.47,2022-01-01 04:00:00+00:00,-1
...,...,...,...
163,239.02,2022-01-07 19:00:00+00:00,0
164,223.79,2022-01-07 20:00:00+00:00,0
165,213.85,2022-01-07 21:00:00+00:00,0
166,211.64,2022-01-07 22:00:00+00:00,0
